In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

/Users/thomas/Documents/srt_translate/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/thomas/Documents/srt_translate/.venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [2]:
sample_text = "all childish, aren't you?"
batch = tokenizer([sample_text], return_tensors="pt")

generated_ids = model.generate(**batch)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

/Users/thomas/Documents/srt_translate/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"Tout enfantin, n'est-ce pas?"

In [3]:
f = open("subtitles_three_body_10.srt", "r+")

FileNotFoundError: [Errno 2] No such file or directory: 'subtitles_three_body_10.srt'

In [9]:
f = open("./translated.srt", "r+")

In [10]:
f.read().split("\n")

['1',
 '00:00:16,870 --> 00:00:21,000',
 'Ji Chang, ça fait huit nuits.',
 '',
 '2',
 '00:00:21,000 --> 00:00:23,280',
 'On est dans une autre ère chaotique.',
 '',
 '3',
 '00:00:23,280 --> 00:00:27,800',
 "Le soleil ne s'est plus jamais levé.",
 '',
 '4',
 '00:00:27,800 --> 00:00:30,760',
 "Mon roi, c'est temporaire.",
 '',
 '5',
 '00:00:30,760 --> 00:00:32,560',
 'Le soleil se lèvera bientôt.',
 '',
 '6',
 '00:00:32,560 --> 00:00:37,240',
 "L'ère stable et",
 'le printemps reprendront.',
 '',
 '7',
 '00:00:39,470 --> 00:00:42,470',
 'Chauffer le quadripode.',
 '',
 '8',
 '00:00:46,070 --> 00:00:47,840',
 'Mon roi!',
 '',
 '9',
 '00:00:47,840 --> 00:00:48,840',
 'Il y a...',
 '',
 '10',
 '00:00:48,840 --> 00:00:53,070',
 'Il y a trois étoiles volantes dans le ciel!',
 '',
 '11',
 '00:00:54,840 --> 00:01:00,670',
 'Tu ne sais pas ce que ça veut dire',
 "d'avoir trois étoiles volantes dans le ciel, je suppose?",
 '',
 '12',
 '00:01:00,670 --> 00:01:05,760',
 "Ça veut dire qu'un hiver ru

In [19]:
source_srt = f.readlines()
f.close()

In [20]:
extract = source_srt

In [21]:
text_lines = []
dico_lines = {}
full_text_lines = []

for i, line in enumerate(extract):
    full_text_lines.append(line)
    if "-->" not in line:
        text_lines.append(line)
        dico_lines[line] = i

In [22]:
aggregated_text_lines = []
aggregated_dico_lines = {}
seen_i = set()
n = len(text_lines)

for i, line in enumerate(text_lines):
    if i in seen_i:
        continue
    agg_line = "
    agg_dico = []
    if i<n-1 and text_lines[i+1] == "\n":
        #print("### 1 ###")
        #print(i,line)
        agg_line += line
        agg_dico.append(dico_lines[line])
        seen_i.add(i)
        aggregated_text_lines.append(agg_line)
        aggregated_dico_lines[agg_line] = agg_dico
    if i>0 and i<n-2 and text_lines[i+2] == "\n" and text_lines[i-1] != "\n":
        #print("### 2 ###")
        #print(i,line)
        #print(i+1,text_lines[i+1])
        seen_i.add(i)
        seen_i.add(i+1)
        agg_line += line + " " + text_lines[i+1]
        agg_dico.append(dico_lines[line])
        agg_dico.append(dico_lines[text_lines[i+1]])
        aggregated_text_lines.append(text_lines[i+1])
        aggregated_dico_lines[agg_line] = agg_dico

In [23]:
for full_line_text, line_pos in aggregated_dico_lines.items():
    sample_text = full_line_text.replace("\n", "").replace("â™ª", "") # pour le symbole note de musique
    batch = tokenizer([sample_text], return_tensors="pt")
    generated_ids = model.generate(**batch)
    translated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    if len(line_pos) == 1:
        #print(line_pos[0], translated_text)
        full_text_lines[line_pos[0]] = translated_text
    if len(line_pos) == 2:
        split_text = translated_text.split(" ")
        mid_line = len(split_text)//2
        first_line = " ".join(split_text[:mid_line]) + "\n"
        second_line = " ".join(split_text[mid_line:])
        print(line_pos[0], first_line)
        print(line_pos[1], second_line)
        full_text_lines[line_pos[0]] = first_line
        full_text_lines[line_pos[1]] = second_line

38 Qui est impatient de remplir

39 ses yeux du ciel étoilé
55 Appeler humblement pour le

56 vaste espace et le temps
76 Je t'ai dit que tout ce

77 qui est assez bizarre doit être louche.
81 Si vous me le demandez, c'est vraiment

82 comme un schéma pyramidal que j'ai craqué avant.
102 Notre seule conclusion

103 concerne Yang Dong.
107 Yang Dong ne s'est

108 pas suicidée quand elle devait.
112 Mais elle l'a fait

113 quand elle semblait peu probable.
121 Nous ne pouvons faire une percée

122 qu'après avoir trouvé le point clé.
138 Nous devons savoir en détail

139 ce que Yang Dong a fait
155 Ce n'est pas inapproprié pour

156 moi de parler de Yang Dong.
192 Je les ai

193 organisés dans l'ordre chronologique.
197 Quant au contact de Yang

198 Dong et de ses collègues
202 Avec d'autres

203 personnes, j'enquête toujours.
211 En général, les mariages sont

212 très importants pour les femmes.
220 (Ding Yi est sorti acheter des

221 invitations) choisir ses propres invitations de mari

In [25]:
with open('translated.txt', 'w', encoding='utf-8') as file:
    #file.writelines(s + '\n' for s in full_text_lines)
    for line in full_text_lines:
        if line == "\n":
            file.write("\n")
            file.write("\n")
        else:
            file.write(line)

In [16]:
aggregated_text_lines[:100]

['â™ªEndless driftingâ™ª\n',
 'â™ªTo the end of the dark forestâ™ª\n',
 'â™ªThe wind is like a handâ™ª\n',
 'â™ªTrying to keep timeâ™ª\n',
 'â™ªWho is the hunter with a gunâ™ª\n',
 'â™ªTrying to devour the entire universeâ™ª\n',
 'â™ªThe light is like an eyeâ™ª\n',
 'â™ªThat penetrates dreamsâ™ª\n',
 "â™ªDon't answerâ™ª\n",
 'with the starry skyâ™ª\n',
 'â™ªThe earth is no longer rotatingâ™ª\n',
 'â™ªYou gave me a small universeâ™ª\n',
 'â™ªIn the arms of the universeâ™ª\n',
 'the vast space and timeâ™ª\n',
 'â™ªThe world aches dullyâ™ª\n',
 '=Three-Body=\n',
 '=Episode 10=\n',
 'Lord or God, neither is related.\n',
 'must be fishy.\n',
 'a pyramid scheme I cracked before.\n',
 'The mental state is similar.\n',
 'Shen Yufei must have been brainwashed.\n',
 "I think Yang Dong's death...\n",
 'Yes.\n',
 'is concerning Yang Dong.\n',
 'when she should.\n',
 'unlikely to do it.\n',
 'What happened during that period?\n',
 'after we find the key point.\n',
 'I think we should see Ding Yi.\n

In [13]:
text_lines[58:61]

['must be fishy.\n', '\n', '20\n']